### Test speed of functions

In [1]:
class TestClass:
    def __init__(self):
        self.AGENT_TEAMS = {
            0: 0,
            1: 0,
            2: 1,
            3: 1
        }

        self.CAPTURE_POSITIONS = {
            0: (1, 1),
            1: (8, 8)
        }

    def test_function1(self, agent_idx):
        """
        Create variables from attributes.
        """
        agent_team = self.AGENT_TEAMS[agent_idx]
        agent_capture_pos = self.CAPTURE_POSITIONS[agent_team]
        return agent_capture_pos

    def test_function2(self, agent_idx):
        """
        Mixture.
        """
        agent_team = self.AGENT_TEAMS[agent_idx]
        return self.CAPTURE_POSITIONS[agent_team]

    def test_function3(self, agent_idx):
        """
        Reference attributes.
        """
        return self.CAPTURE_POSITIONS[self.AGENT_TEAMS[agent_idx]]

In [2]:
tester = TestClass()

In [7]:
%%timeit
tester.test_function1(0)

216 ns ± 6.83 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
%%timeit
tester.test_function2(0)

203 ns ± 3.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
%%timeit
tester.test_function3(0)

200 ns ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
